In [1]:
import os
import getpass


os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

Add tools later

In [2]:
#from tools.design_retriever import DesignRetrieverTool
from chains.design_rag import DesignRAG

# Initialize DesignRAG and create the tool
design_rag = DesignRAG()
#design_retriever = DesignRetrieverTool(rag=design_rag)

test_requirements = {
    "I want a design that is vintage and classic, something easy to use that a grandmother would love"
    }

# Test the retriever
async def test_rag():
    print("Testing RAG retriever with requirements:")
    print("\nRetrieved Designs:")
    print("----------------------------------------")
    
    results = await design_rag.query_similar_designs(test_requirements, 2)
    print(results)

# Run the test
await test_rag()


Loaded 82 design documents
Testing RAG retriever with requirements:

Retrieved Designs:
----------------------------------------
Generated query: "vintage classic easy to use grandmother love design"
Design 180:
Description: This design employs a vintage newspaper aesthetic with a classic serif typography that evokes an old-world charm, utilizing sepia-toned paper backgrounds to enhance its nostalgic feel. The layout is text-heavy with a deliberate obfuscation, reflecting a layered collage effect. Its balanced placement keeps the focus central, inviting closer inspection and interaction.
Categories: Vintage, Nostalgic, Typography, Collage, Editorial
Visual Characteristics: Sepia tone, Serif typography, Textured background, Layered elements, Central focus
URL: https://csszengarden.com/180

Design 182:
Description: The design creatively utilizes a retro theme with vinyl records as the prominent visual element to evoke a sense of nostalgia and classic style, complemented by a muted green 

Pick a model good for chat and tools

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o", 
    temperature=0,
    streaming=True
)

model.bind_tools(tool_belt)

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1245518d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x124548e50>, root_client=<openai.OpenAI object at 0x1108f9310>, root_async_client=<openai.AsyncOpenAI object at 0x115d92090>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True), kwargs={'tools': [{'type': 'function', 'function': {'name': 'design_retriever', 'description': 'Retrieves similar designs based on style requirements', 'parameters': {'properties': {'requirements': {'type': 'object'}, 'num_examples': {'default': 3, 'type': 'integer'}}, 'required': ['requirements'], 'type': 'object'}}}]}, config={}, config_factories=[])

Initialize state


In [12]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Set up the nodes and graph


In [16]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="""You are a helpful design assistant that can retrieve and analyze design examples. 
When a user describes their design preferences or requirements, use the design_retriever tool to find relevant examples.

Always use the design_retriever tool when:
- A user describes specific design requirements
- A user asks to see similar designs
- You need to find design inspiration based on user preferences

Format the requirements as a dictionary with these keys:
- style_description: Brief description of desired visual style
- key_elements: List of important visual elements
- color_scheme: Description of colors
- layout_preferences: Layout requirements
- mood: Desired emotional impact
""")

def call_model(state):
  messages = [system_message] + state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)
uncompiled_graph.set_entry_point("agent")


def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
  "agent",
  should_continue
)
uncompiled_graph.add_edge("action", "agent")

graph = uncompiled_graph.compile()

#formatted chain

def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

graph_chain = convert_inputs | graph | parse_output



Try it out!

In [ ]:
from langchain_core.messages import HumanMessage

async for chunk in graph.astream({"messages" : [HumanMessage(content="Hello, how are you?")]}, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content="Hello! I'm here and ready to help you with any design needs or questions you might have. How can I assist you today?", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf'}, id='run-4edce0b5-fdec-4d5d-a4a6-92430faca51a-0')]





Let's see if the RAG tool works.

In [18]:
# Create a test message
from langchain_core.messages import HumanMessage

test_message = HumanMessage(
    content="""I want to see a design matching this description: 
    I want it to use a monochromatic color scheme with subtle accent colors. 
    The layout should be grid-based with clear hierarchy. 
    The overall mood should be professional and sophisticated."""
)

async for chunk in graph.astream({"messages" : [test_message]}, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content="To find a design that matches your description, I'll use the design_retriever tool. Here are the requirements based on your description:\n\n- style_description: Monochromatic with subtle accents\n- key_elements: Grid-based layout, clear hierarchy\n- color_scheme: Monochromatic with subtle accent colors\n- layout_preferences: Grid-based\n- mood: Professional and sophisticated\n\nLet's find some examples for you.", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f9f4fb6dbf'}, id='run-8fa2e4af-671c-4c75-82fd-a7b3d6237e54-0')]



